Here we can find correlatins of biotech stocks data, correlated pairs can be used for arbitrage trading strategy. 

In [ ]:
import numpy as np
import pandas as pd
!pip install yfinance
!pip install openpyxl

In [ ]:
#scraping historical data
import yfinance as yf
historical_data={}
biotech_tickers_nasdaq=pd.read_excel('../input/completelistofforeignbiotechstocks/Complete-List-of-Biotech-Stocks-on-NASDAQ-Oct-3-2021.xlsx')['Ticker'][:827]
for i in biotech_tickers_nasdaq:
  ticker = yf.Ticker(i)
  hist = ticker.history(period="max")
  historical_data[i]=hist.reset_index()

In [ ]:
#correlation swing (+0.1 -0.1 )
corr=0.1
#ratio of days when correlation observed (90% by default)
cases_ratio=0.9

tickers=list(biotech_tickers_nasdaq)
correlation_list=[]
for i in tickers:
    df=historical_data[i].copy()
    tickeers_rem=tickers.copy()
    tickeers_rem.remove(i)
    for a in tickeers_rem: 
        df2=historical_data[a].copy()
        df2.columns=["Close_2" if df2.columns[i]=="Close" else df2.columns[i] for i in range(len(df2.columns)) ]
        df3=pd.DataFrame()
        df3=pd.merge(df, df2, on=['Date'], how='inner')
        df3['koef']=df3['Close']/df3["Close_2"]
        df3['Test']=[i<1+corr and i>1-corr for i in df3['koef']]
        if df3['Test'].sum()>cases_ratio*len(df3['Test']):
            block=[a,i,df3['Test'].sum(),cases_ratio*len(df3['Test'])]
            correlation_list.append(block)

In [ ]:
correlation_list